In [1]:
import json
import re
from abc import ABC, abstractmethod
from collections import Counter, defaultdict
from dataclasses import dataclass
from datetime import datetime
from functools import lru_cache, reduce
from itertools import combinations, count, cycle, permutations, product, takewhile
from operator import mul, or_
from queue import PriorityQueue
from statistics import mean, median, mode
from typing import List, Optional, Tuple

import numpy as np
import z3
from bitstring import BitStream
from more_itertools import windowed
from numba import jit, njit
from numpy.linalg import matrix_power
from PIL import Image
from pyparsing import Forward, ParseException
from sympy import Quaternion

### Day 1

In [2]:
with open("day1.txt") as f:
    depths = [int(s) for s in f.read().splitlines()]

In [3]:
sum(1 for a, b in windowed(depths, 2) if a < b)

1298

In [4]:
three_sums = (sum(w) for w in windowed(depths, 3))
sum(1 for a, b in windowed(three_sums, 2) if a < b)

1248

### Day 2

In [5]:
with open("day2.txt") as f:
    instructions = f.read().splitlines()

In [6]:
x = y = 0
for instruction in instructions:
    word, steps = instruction.split()
    steps = int(steps)
    if word == "forward":
        x += steps
    elif word == "down":
        y += steps
    elif word == "up":
        y -= steps
x * y

1815044

In [7]:
x = y = aim = 0
for instruction in instructions:
    word, steps = instruction.split()
    steps = int(steps)
    if word == "forward":
        x += steps
        y += aim * steps
    elif word == "down":
        aim += steps
    elif word == "up":
        aim -= steps
x * y

1739283308

### Day 3

In [8]:
with open("day3.txt") as f:
    reports = [[c == "1" for c in line] for line in f.read().splitlines()]

In [9]:
counts = [Counter(report[i] for report in reports) for i in range(12)]
gamma = reduce(lambda a, t: 2 * a + t, (count[1] > count[0] for count in counts))
epsilon = reduce(lambda a, t: 2 * a + t, (count[1] < count[0] for count in counts))
gamma * epsilon

3958484

In [10]:
chosen, i = reports.copy(), 0
while len(chosen) > 1:
    counter = Counter(report[i] for report in chosen)
    c = counter[1] >= counter[0]
    chosen, i = [report for report in chosen if report[i] == c], i + 1
ogr = reduce(lambda a, t: 2 * a + t, chosen[0])
chosen, i = reports.copy(), 0
while len(chosen) > 1:
    counter = Counter(report[i] for report in chosen)
    c = counter[1] < counter[0]
    chosen, i = [report for report in chosen if report[i] == c], i + 1
csr = reduce(lambda a, t: 2 * a + t, chosen[0])
ogr * csr

1613181

### Day 4

In [11]:
with open("day4.txt") as f:
    calls = [int(s) for s in f.readline()[:-1].split(",")]
    f.readline()
    boards = f.read()[:-1].split("\n\n")
    boards = [[list(map(int, line.split())) for line in b.splitlines()] for b in boards]
    boards = np.array(boards, np.int8)

In [12]:
def play():
    games = boards.copy()
    for call in calls:
        games[games == call] = -1
        for game in games:
            if any(all(game[i, :] == -1) or all(game[:, i] == -1) for i in range(5)):
                return call, game

In [13]:
call, game = play()
sum(game[game > 0]) * call

58838

In [14]:
def play():
    games = boards.copy()
    done = set()
    for call in calls:
        games[games == call] = -1
        for gi, game in enumerate(games):
            if any(all(game[i, :] == -1) or all(game[:, i] == -1) for i in range(5)):
                done.add(gi)
            if len(done) == len(games):
                return call, game

In [15]:
call, game = play()
sum(game[game > 0]) * call

6256

### Day 5

In [16]:
with open("day5.txt") as f:
    instructions = f.read().splitlines()
    lines = []
    for instr in instructions:
        m = re.match(r"(\d+),(\d+) -> (\d+),(\d+)", instr)
        lines.append(list(map(int, m.groups())))

In [17]:
def points():
    for x1, y1, x2, y2 in lines:
        if x1 == x2:
            for y in range(min(y1, y2), 1 + max(y1, y2)):
                yield x1, y
        elif y1 == y2:
            for x in range(min(x1, x2), 1 + max(x1, x2)):
                yield x, y1

In [18]:
sum(1 for point, count in Counter(points()).items() if count > 1)

7473

In [19]:
def points():
    for x1, y1, x2, y2 in lines:
        if x2 < x1:
            x1, y1, x2, y2 = x2, y2, x1, y1
        sgn = 1 if y2 > y1 else -1 if y2 < y1 else 0
        if x1 == x2:
            for y in range(y1, y2 + sgn, sgn):
                yield x1, y
        else:
            for x in range(x1, 1 + x2):
                yield x, y1 + sgn * (x - x1)

In [20]:
sum(1 for point, count in Counter(points()).items() if count > 1)

24164

### Day 6

In [21]:
with open("day6.txt") as f:
    counter = Counter(map(int, f.read()[:-1].split(",")))
    fish_with_timer = [counter[i] for i in range(9)]

In [22]:
def pass_days(fish_with_timer, n_days):
    for _ in range(n_days):
        zeros = fish_with_timer[0]
        fish_with_timer = fish_with_timer[1:] + [zeros]
        fish_with_timer[6] += zeros
    return fish_with_timer

In [23]:
sum(pass_days(fish_with_timer.copy(), 80))

351092

In [24]:
sum(pass_days(fish_with_timer.copy(), 256))

1595330616005

### Day 7

In [25]:
with open("day7.txt") as f:
    positions = list(map(int, f.read()[:-1].split(",")))

In [26]:
q = median(positions)
candidates = (int(q), int(q) + 1)
min(sum(abs(n - c) for n in positions) for c in candidates)

344535

In [27]:
q = mean(positions)
candidates = (int(q), int(q) + 1)
min(sum(d * (d + 1) // 2 for d in (abs(n - c) for n in positions)) for c in candidates)

95581659

### Day 8

In [28]:
with open("day8.txt") as f:
    entries = [[w.split() for w in s.split(" | ")] for s in f.read().splitlines()]

In [29]:
segments = "abcefg cf acdeg acdfg bcdf abdfg abdefg acf abcdefg abcdfg".split()

In [30]:
s1478 = [len(segments[i]) for i in (1, 4, 7, 8)]
sum(1 for entry in entries for w in entry[1] if len(w) in s1478)

261

In [31]:
def permute(word, permutation):
    return "".join(sorted(permutation[ord(c) - ord("a")] for c in word))


def solve(entry):
    for p in permutations("abcdefg"):
        if all(permute(w, p) in segments for w in entry[0] + entry[1]):
            return int("".join(str(segments.index(permute(w, p))) for w in entry[1]))

In [32]:
sum(solve(entry) for entry in entries)

987553

### Day 9

In [33]:
with open("day9.txt") as f:
    heightmap = np.array([[int(c) for c in s] for s in f.read().splitlines()], np.int8)

In [34]:
h, w = heightmap.shape
ehm = np.full((h + 2, w + 2), 10, dtype=np.int8)
ehm[1:-1, 1:-1] = heightmap
lowpoints = np.ones(heightmap.shape, np.bool_)
for di, dj in ((-1, 0), (1, 0), (0, -1), (0, 1)):
    lowpoints &= ehm[1:-1, 1:-1] < ehm[1 + di : h + 1 + di, 1 + dj : w + 1 + dj]

In [35]:
sum(1 + heightmap[lowpoints])

633

In [36]:
@lru_cache(maxsize=None)
def basin(i, j):
    if heightmap[i, j] == 9 or lowpoints[i, j]:
        return i, j
    for di, dj in ((-1, 0), (1, 0), (0, -1), (0, 1)):
        if ehm[1 + i + di, 1 + j + dj] < heightmap[i, j]:
            return basin(i + di, j + dj)

In [37]:
counter = Counter(basin(i, j) for i, j in np.ndindex(heightmap.shape))
reduce(mul, sorted(counter.values())[-3:])

1050192

### Day 10

In [38]:
with open("day10.txt") as f:
    nslines = f.read().splitlines()

In [39]:
chunk = Forward()
chunks = chunk * (0,)
chunk <<= reduce(or_, (s[0] + chunks + s[1] for s in "() <> [] {}".split()))
good = Forward()
incomplete = reduce(or_, (c + good for c in "(<[{"))
good <<= chunks + incomplete[0, 1]

In [40]:
def first_illegals():
    for line in nslines:
        try:
            good.parse_string(line, parse_all=True)
        except ParseException as pe:
            yield line[pe.col - 1]

In [41]:
sum((3, 57, 1197, 25137)[")]}>".index(c)] for c in first_illegals())

268845

In [42]:
def completion(line):
    res = ""
    while not chunks.matches(line + res, parse_all=True):
        res += next(c for c in ")]}>" if good.matches(line + res + c))
    return res

In [43]:
def score(completion):
    return reduce(lambda a, c: 5 * a + ")]}>".index(c) + 1, completion, 0)

In [44]:
median(score(completion(line)) for line in nslines if good.matches(line))

4038824534

### Day 11

In [45]:
with open("day11.txt") as f:
    initial_levels = [list(map(int, s)) for s in f.read().splitlines()]
    initial_levels = np.array(initial_levels, np.int8)

In [46]:
def step(data):
    flashes, flashers, h, w = 0, -1, *data.shape
    has_flashed = np.zeros(data.shape, np.bool_)
    data += 1
    while flashers:
        will_flash = (data > 9) & ~has_flashed
        energy = np.zeros((h + 2, w + 2), dtype=np.int8)
        for di, dj in product((-1, 0, 1), repeat=2):
            energy[1 + di : h + 1 + di, 1 + dj : w + 1 + dj] += will_flash
        data += energy[1:-1, 1:-1] - will_flash
        has_flashed |= will_flash
        flashers = will_flash.sum()
        flashes += flashers
    data[data > 9] = 0
    return flashes

In [47]:
levels = initial_levels.copy()
sum(step(levels) for _ in range(100))

1667

In [48]:
levels = initial_levels.copy()
octopuses = reduce(mul, levels.shape)
step_number = 1
while step(levels) != octopuses:
    step_number += 1
step_number

488

### Day 12

In [49]:
with open("day12.txt") as f:
    entries = f.read().splitlines()

In [50]:
adj = defaultdict(list)
for entry in entries:
    a, b = re.match(r"(.+)-(.+)", entry).groups()
    adj[a].append(b)
    adj[b].append(a)

In [51]:
smalls = [s for s in adj.keys() if s.islower()]

In [52]:
ways = defaultdict(lambda: defaultdict(int))
for src in smalls:
    for dst in adj[src]:
        if dst in smalls:
            ways[src][dst] += 1
        else:
            for dst2 in adj[dst]:
                ways[src][dst2] += 1

In [53]:
@lru_cache(maxsize=None)
def paths(src="start", seen=("start",)):
    if src == "end":
        return 1
    result = 0
    for dst, count in ways[src].items():
        if dst not in seen:
            result += count * paths(dst, tuple(sorted(seen + (dst,))))
    return result

In [54]:
paths()

5212

In [55]:
@lru_cache(maxsize=None)
def paths(src="start", seen=("start",), doubled=False):
    if src == "end":
        return 1
    result = 0
    for dst, count in ways[src].items():
        if dst not in seen:
            result += count * paths(dst, tuple(sorted(seen + (dst,))), doubled)
        elif not doubled and dst != "start":
            result += count * paths(dst, seen, True)
    return result

In [56]:
paths()

134862

### Day 13

In [57]:
with open("day13.txt") as f:
    coords, fold_lines = f.read().split("\n\n")
    coords = [list(map(int, line.split(",")))[::-1] for line in coords.splitlines()]
    h, w = (max(ij[d] for ij in coords) + 1 for d in (0, 1))
    folds = []
    for line in fold_lines.splitlines():
        d, n = re.match(r".+(x|y)=(.+)", line).groups()
        folds.append(("yx".index(d), int(n)))

In [58]:
stars = np.zeros((h, w), np.bool_)
for i, j in coords:
    stars[i, j] = True

In [59]:
def fold(d, n, folded):
    overlap = min(n, folded.shape[d] - n - 1)
    if d == 0:
        folded[n - overlap : n, :] |= folded[n + overlap : n : -1, :]
        folded = folded[: overlap + 1, :]
    else:
        folded[:, n - overlap : n] |= folded[:, n + overlap : n : -1]
        folded = folded[:, : overlap + 1]
    return folded

In [60]:
fold(*folds[0], stars.copy()).sum()

785

In [61]:
folded = reduce(lambda sheet, f: fold(*f, sheet), folds, stars.copy())

In [62]:
Image.fromarray(folded).resize(n * 4 for n in reversed(folded.shape))

### Day 14

In [63]:
with open("day14.txt") as f:
    template, rule_lines = f.read().split("\n\n")

In [64]:
letters = sorted(set(rule_lines).difference(" ->\n"))
n_letters = len(letters)

In [65]:
pairs = np.zeros((n_letters ** 2, 1), dtype=np.int_)
for a, b in windowed((letters.index(c) for c in template), 2):
    pairs[n_letters * a + b] += 1

In [66]:
rules = np.zeros((n_letters ** 2,) * 2, dtype=np.int_)
for line in rule_lines.splitlines():
    left, right, insert = re.match(r"(.)(.) -> (.)", line).groups()
    left, right, insert = (letters.index(c) for c in (left, right, insert))
    pair = n_letters * left + right
    rules[n_letters * left + insert, pair] += 1
    rules[n_letters * insert + right, pair] += 1

In [67]:
def answer14(steps):
    counts = (matrix_power(rules, steps) @ pairs).reshape((n_letters,) * 2)
    counts = [sum(counts[i, :] + counts[:, i]) for i in range(n_letters)]
    for c in (template[0], template[-1]):
        counts[letters.index(c)] += 1
    counts = [n // 2 for n in counts]
    return max(counts) - min(counts)

In [68]:
answer14(10)

3342

In [69]:
answer14(40)

3776553567525

### Day 15

In [70]:
with open("day15.txt") as f:
    grid = [list(map(int, line)) for line in f.read().splitlines()]
    grid = np.array(grid, np.int8)

In [71]:
def answer15(grid):
    h, w = grid.shape
    distances = np.zeros(grid.shape, np.int16)
    queue, d = [[(0, 0)]] + [[] for _ in range((h + w) * 9)], 0
    while distances[-1, -1] == 0:
        for i, j in queue[d]:
            if distances[i, j] == d:
                for ii, jj in ((i - 1, j), (i + 1, j), (i, j - 1), (i, j + 1)):
                    if 0 <= ii < h and 0 <= jj < w:
                        if distances[ii, jj] == 0:
                            distances[ii, jj] = dd = grid[ii, jj] + d
                            queue[dd].append((ii, jj))
        d += 1
    return distances[-1, -1]

In [72]:
answer15(grid)

447

In [73]:
grid5 = []
h, w = grid.shape
for i in range(5 * w):
    row = [(grid[i % h][j % h] + i // h + j // w - 1) % 9 + 1 for j in range(5 * h)]
    grid5.append(row)
answer15(np.array(grid5, dtype=np.int8))

2825

### Day 16

In [74]:
@dataclass
class Packet(ABC):
    version: int
    tid: int

    @property
    def version_sum(self) -> int:
        return self.version

    @property
    @abstractmethod
    def value(self) -> int:
        pass

    @staticmethod
    def from_bitstream(bits) -> ("Packet", int):
        version = bits.read("uint:3")
        tid = bits.read("uint:3")
        bits_read = 6
        if tid == 4:
            n = 0
            while bits.read("bool"):
                n = (n << 4) + bits.read("uint:4")
                bits_read += 5
            n = (n << 4) + bits.read("uint:4")
            bits_read += 5
            return Literal(version, tid, n), bits_read
        packets = []
        ltid = bits.read("uint:1")
        bits_read += 1
        if ltid == 0:
            tlib = bits.read("uint:15")
            bits_read += 15
            while tlib:
                packet, length = Packet.from_bitstream(bits)
                packets.append(packet)
                tlib -= length
                bits_read += length
        else:
            nos = bits.read("uint:11")
            bits_read += 11
            for _ in range(nos):
                packet, length = Packet.from_bitstream(bits)
                packets.append(packet)
                bits_read += length
        return Operator(version, tid, packets), bits_read


@dataclass
class Literal(Packet):
    n: int

    @property
    def value(self) -> int:
        return self.n


@dataclass
class Operator(Packet):
    packets: List[Packet]

    @property
    def version_sum(self) -> int:
        return super().version_sum + sum(p.version_sum for p in self.packets)

    @property
    def value(self) -> int:
        if self.tid == 0:
            return sum(p.value for p in self.packets)
        elif self.tid == 1:
            return reduce(mul, (p.value for p in self.packets), 1)
        elif self.tid == 2:
            return min(p.value for p in self.packets)
        elif self.tid == 3:
            return max(p.value for p in self.packets)
        elif self.tid == 5:
            return int(self.packets[0].value > self.packets[1].value)
        elif self.tid == 6:
            return int(self.packets[0].value < self.packets[1].value)
        elif self.tid == 7:
            return int(self.packets[0].value == self.packets[1].value)

In [75]:
with open("day16.txt") as f:
    packet = Packet.from_bitstream(BitStream(hex=f.read()[:-1]))[0]

In [76]:
packet.version_sum

847

In [77]:
packet.value

333794664059

### Day 17

In [78]:
with open("day17.txt") as f:
    data = f.read()[:-1]
    match = re.match(r"target area: x=(.+)\.\.(.+), y=(.+)\.\.(.+)", data)
    x0, x1, y0, y1 = map(int, match.groups())

In [79]:
def steps(dx, dy):
    x = y = 0
    while True:
        x, y, dx, dy = x + dx, y + dy, dx, dy - 1
        dx = dx - 1 if dx > 0 else 0 if dx == 0 else dx + 1
        yield x, y

In [80]:
def hits(dxdy):
    path = takewhile(lambda xy: xy[0] <= x1 and xy[1] >= y0, steps(*dxdy))
    return any(x0 <= x <= x1 and y0 <= y <= y1 for x, y in path)


dxdys = filter(hits, product(range(1, 1 + x1), range(1 + y1 // x1 + (x1 + 1) // 2)))
_, dy = max(dxdys, key=lambda dxdy: dxdy[1])
dy * (dy + 1) // 2

6555

In [81]:
dxdys = filter(hits, product(range(1, 1 + x1), range(y0, 1 + y1 // x1 + (x1 + 1) // 2)))
sum(1 for _ in dxdys)

4973

### Day 18

In [82]:
with open("day18.txt") as f:
    sf_numbers = [json.loads(line) for line in f.read().splitlines()]

In [83]:
def addleft(x, n):
    return x + n if type(x) == int else (addleft(x[0], n), x[1])


def addright(x, n):
    return x + n if type(x) == int else (x[0], addright(x[1], n))

In [84]:
def explode(x, depth=0):
    if type(x) != int:
        if depth == 4:
            return 0, x
        e, (a, b) = explode(x[0], depth + 1)
        if a is not None:
            return (e, addleft(x[1], b)), (a, 0)
        e, (a, b) = explode(x[1], depth + 1)
        if b is not None:
            return (addright(x[0], a), e), (0, b)
    return x, (None, None)

In [85]:
def split(x):
    if type(x) != int:
        s, c = split(x[0])
        if c:
            return (s, x[1]), True
        s, c = split(x[1])
        if c:
            return (x[0], s), True
    elif x >= 10:
        return (x // 2, (x + 1) // 2), True
    return x, False

In [86]:
def sf_add(*x):
    a = True
    while a:
        while a is not None:
            x, (a, b) = explode(x)
        x, a = split(x)
    return x

In [87]:
def mag(x):
    return x if type(x) == int else 3 * mag(x[0]) + 2 * mag(x[1])

In [88]:
mag(reduce(sf_add, sf_numbers))

3987

In [89]:
max(mag(sf_add(a, b)) for a, b in permutations(sf_numbers, 2))

4500

### Day 19

In [90]:
with open("day19.txt") as f:
    scans = []
    for scanner in f.read().split("\n\n"):
        points = scanner.splitlines()[1:]
        scan = [np.fromiter(map(int, line.split(",")), np.int16) for line in points]
        scans.append(scan)

In [91]:
def overlap(sa, sb):
    fps = [set(tuple(b - a) for a, b in permutations(s, 2)) for s in (sa, sb)]
    return len(fps[0] & fps[1]) >= 132

In [92]:
rots = set()
ijk = [Quaternion(1, *(0,) * d, 1) for d in range(3)]
for es in product(range(4), repeat=3):
    q = reduce(mul, (b ** e for b, e in zip(ijk, es)), Quaternion(1))
    rots.add(tuple(q.to_rotation_matrix()))
rots = [np.array(r, dtype=np.int8).reshape((3, 3)) for r in rots]

In [93]:
locations = [None] * len(scans)
used_indices = []


def points(si, f=lambda p: p):
    used_indices.append(si)
    locations[si] = f((0, 0, 0))
    result = set(map(tuple, scans[si]))
    for j, sj in enumerate(scans):
        if j not in used_indices:
            for r in rots:
                if overlap(scans[si], rsj := [r @ p for p in sj]):
                    d = mode(tuple(bb - aa) for aa, bb in product(scans[si], rsj))
                    tsj = (r @ p - d for p in points(j, lambda p: f(r @ p - d)))
                    result.update(map(tuple, tsj))
                    break
    return result

In [94]:
len(points(0))

394

In [95]:
max(sum(np.abs(a - b)) for a, b in combinations(locations, 2))

12304

### Day 20

In [96]:
with open("day20.txt") as f:
    iea, ii = f.read().split("\n\n")
    iea = [c == "#" for c in iea]
    ii = np.array([[c == "#" for c in line] for line in ii.splitlines()], np.bool_)
    uiea = np.vectorize(lambda n: iea[n])

In [97]:
def step(ii, fill):
    h, w = ii.shape
    ii = ~ii if fill else ii
    ii2 = np.zeros((h + 2, w + 2), dtype=np.int16)
    for di, dj in product((2, 1, 0), repeat=2):
        ii2 <<= 1
        ii2[di : di + h, dj : dj + w] |= ii
    ii2 = 511 ^ ii2 if fill else ii2
    return uiea(ii2), iea[(0, 511)[fill]]

In [98]:
ic, fill = ii.copy(), 0
ic, fill = step(ic, fill)
ic, fill = step(ic, fill)
ic.sum()

5097

In [99]:
ic, fill = ii.copy(), 0
for _ in range(50):
    ic, fill = step(ic, fill)
ic.sum()

17987

### Day 21

In [100]:
with open("day21.txt") as f:
    initial_positions = tuple(int(line[-1]) for line in f.read().splitlines())

In [101]:
def play():
    positions = list(initial_positions)
    die = cycle(range(1, 101))
    rolled = 0
    scores = [0, 0]
    while True:
        for player in range(2):
            positions[player] += next(die) + next(die) + next(die)
            rolled += 3
            positions[player] = (positions[player] - 1) % 10 + 1
            scores[player] += positions[player]
            if scores[player] >= 1000:
                return scores[1 - player] * rolled

In [102]:
play()

734820

In [103]:
@lru_cache(maxsize=None)
def wins(positions=initial_positions, scores=(0, 0)):
    if scores[1] >= 21:
        return (0, 1)
    r0 = r1 = 0
    for a, b, c in product(range(1, 4), repeat=3):
        s = a + b + c
        np = (positions[0] + s - 1) % 10 + 1
        ns = scores[0] + np
        w1, w0 = wins((positions[1], np), (scores[1], ns))
        r0, r1 = r0 + w0, r1 + w1
    return (r0, r1)

In [104]:
max(wins())

193170338541590

### Day 22

In [105]:
with open("day22.txt") as f:
    cmds = []
    pattern = r"(on|off) x=(.+)\.\.(.+),y=(.+)\.\.(.+),z=(.+)\.\.(.+)"
    for cmd in f.read().splitlines():
        groups = re.match(pattern, cmd).groups()
        cbounds = np.array([int(g) for g in groups[1:]]).reshape((3, 2))
        cbounds[:, 1] += 1
        cmds.append((groups[0] == "on", cbounds))
    fills, bounds = (np.array(lst) for lst in zip(*cmds))

In [106]:
grid = np.zeros((101,) * 3, np.bool_)
for fill, b in zip(fills, bounds):
    b = b + 50
    grid[tuple(slice(*b[d, :]) for d in range(3))] = fill
grid.sum()

615869

In [107]:
@njit
def cubes(b, fills, bounds, fill=0, i=0):
    for j in range(i, len(fills)):
        cb = bounds[j]
        if ((cb[:, 0] <= b[:, 0]) & (cb[:, 1] >= b[:, 1])).all():
            fill = fills[j]
        elif ((cb[:, 0] < b[:, 1]) & (cb[:, 1] > b[:, 0])).all():
            for d, e in np.ndindex((3, 2)):
                if b[d, 0] < cb[d, e] < b[d, 1]:
                    break
            (b0, b1), b[d, 1] = b[d, :], cb[d, e]
            res = cubes(b, fills, bounds, fill, j)
            b[d, 1], b[d, 0] = b1, cb[d, e]
            res, b[d, 0] = res + cubes(b, fills, bounds, fill, j), b0
            return res
    return (b[:, 1] - b[:, 0]).prod() if fill else 0

In [108]:
box = np.zeros((3, 2), np.int64)
ends = (f(bounds[:, :, e], axis=0) for e, f in enumerate((np.min, np.max)))
box[:, 0], box[:, 1] = ends
cubes(box, fills, bounds)

1323862415207825

### Day 23

In [109]:
with open("day23.txt") as f:
    lines = f.read().splitlines()

In [110]:
def run(lines):
    RL = len(lines) - 3
    rooms = []
    for ri in range(4):
        t = "".join(lines[-2 - i][2 * ri + 3] for i in range(RL))
        t = re.match("ABCD"[ri] + r"*(.+)", t)[1]
        rooms.append(["ABCD".index(c) + 1 for c in t])
    cost = [1, 10, 100, 1000]

    def neighbors(state):
        rls, hall = state
        for ri, rl in enumerate(rls):
            rj = 2 * ri + 2
            if rl == 0:
                for start, stop, step in ((rj - 1, -1, -1), (rj + 1, 11, 1)):
                    for j in range(start, stop, step):
                        if hall[j] != 0:
                            if hall[j] == ri + 1:
                                hall2 = hall[:j] + (0,) + hall[j + 1 :]
                                yield (rls, hall2), cost[ri] * abs(j - rj)
                            break
            else:
                for start, stop, step in ((rj - 1, -1, -1), (rj + 1, 11, 1)):
                    for j in range(start, stop, step):
                        if hall[j] != 0:
                            break
                        elif j not in (2, 4, 6, 8):
                            rls2 = rls[:ri] + (rls[ri] - 1,) + rls[ri + 1 :]
                            atype = rooms[ri][rl - 1]
                            hall2 = hall[:j] + (atype,) + hall[j + 1 :]
                            d = cost[atype - 1] * (abs(j - rj) + RL - (rl - 1))
                            yield (rls2, hall2), d

    def heuristic(state):
        rls, hall = state
        result = sum(cost[v - 1] * abs(i - 2 * v) for i, v in enumerate(hall) if v)
        for ri, (rl, room) in enumerate(zip(rls, rooms)):
            for p, c in enumerate(rooms[ri][:rl]):
                result += cost[c - 1] * (abs(2 * (ri + 1 - c)) + RL - p)
        return result

    pq = PriorityQueue()
    start = (tuple(map(len, rooms)), (0,) * 11)
    base = RL * (RL + 1) // 2 * 1111
    pq.put((base, start))
    gScore = defaultdict(lambda: 1_000_000)
    gScore[start] = base

    while True:
        current = pq.get()[1]
        for n, d in neighbors(current):
            tgs = gScore[current] + d
            if tgs < gScore[n]:
                gScore[n] = tgs
                h = heuristic(n)
                if h == 0:
                    return tgs
                pq.put((tgs + h, n))

In [111]:
run(lines)

14348

In [112]:
insert = """  #D#C#B#A#
  #D#B#A#C#
""".splitlines()
run(lines[:3] + insert + lines[3:])

40954

### Day 24

In [113]:
with open("day24.txt") as f:
    lines = []
    for line in f.read().splitlines():
        lines.append(re.match(r"([^ ]+) (.)(?: (.+))?", line).groups())

In [114]:
def doinstr(instr, state, inp):
    v, r0, r1 = instr
    if r1:
        b = state[r1] if r1 in "wxyz" else int(r1)
    if v == "add":
        state[r0] += b
    elif v == "div":
        state[r0] /= b
    elif v == "eql":
        state[r0] = z3.If(state[r0] == b, 1, 0)
    elif v == "inp":
        state[r0] = inp
    elif v == "mod":
        state[r0] %= b
    elif v == "mul":
        state[r0] *= b
    return state

In [115]:
def partial(start, stop, z_last):
    state = {"z": z_last, **{a: z3.Int(f"{a}_last") for a in "wxy"}}
    for instr in lp[start:stop]:
        state = doinstr(instr, state, z3.Int("input"))
    return z3.simplify(state["z"])

In [116]:
def solve(digitorder):
    solver = z3.Solver()
    z = [0] * 15
    inp = [z3.Int(f"inp_{i}") for i in range(14)]
    for i in range(14):
        if lines[i * 18 + 4][2] == "26":
            z[i + 1] = z[i] / 26
            solver.add(z[i] % 26 == -int(lines[i * 18 + 5][2]) + inp[i])
        else:
            z[i + 1] = z[i] * 26 + int(lines[i * 18 + 15][2]) + inp[i]
    for i in inp:
        solver.add(1 <= i)
        solver.add(i <= 9)
    solver.add(z[14] == 0)
    for i in range(14):
        for v in digitorder:
            solver.push()
            solver.add(inp[i] == v)
            if solver.check() == z3.sat:
                break
            solver.pop()
    m = solver.model()
    res = reduce(lambda a, t: 10 * a + t, (m.eval(inp[i]).as_long() for i in range(14)))
    return res

In [117]:
solve(range(9, 0, -1))

99598963999971

In [118]:
solve(range(1, 10))

93151411711211

### Day 25

In [119]:
with open("day25.txt") as f:
    data = np.array([[ord(c) for c in line] for line in f.read().splitlines()], np.int8)

In [120]:
EMPTY, EAST, SOUTH = (ord(c) for c in ".>v")

In [121]:
for step_count in count(1):
    old = data
    moving = (data == EAST) & (np.roll(data, -1, 1) == EMPTY)
    data = np.where(moving, EMPTY, np.where(np.roll(moving, 1, 1), EAST, data))
    moving = (data == SOUTH) & (np.roll(data, -1, 0) == EMPTY)
    data = np.where(moving, EMPTY, np.where(np.roll(moving, 1, 0), SOUTH, data))
    if (data == old).all():
        break
step_count

360